In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
base_path = Path().cwd().parent.parent

In [3]:
with open(base_path / 'data' / 'external' / 'jet2_data.json') as f:
    jet2_json = json.load(f)

In [4]:
jet2 = pd.DataFrame(jet2_json['Data'])
jet2 = jet2[jet2['isEnabledForBooking'] == True]
jet2 = jet2[jet2['destinationIataCodes'] != '']

In [5]:
jet2.head()

,country,label,code,isDepartureAirport,isDestinationAirport,destinationIataCodes,searchTerms,isEnabledForBooking,airportUrlKey
2,Spain,Alicante ALC,ALC,True,True,BHX|MAN|NCL|LBA|GLA|BFS|EMA|STN|EDI,"[ALC, Alicante, ALC, Benidorm, Costa Blanca, S...",True,alicante
3,Spain,Almeria LEI,LEI,True,True,BHX|STN|MAN|GLA|NCL|EMA|LBA,"[LEI, Almeria, LEI, Spain]",True,almeria
4,Netherlands,Amsterdam AMS,AMS,True,True,LBA,"[AMS, Amsterdam, AMS, Holland, Netherlands, Ne...",True,amsterdam
5,Turkey,Antalya AYT,AYT,True,True,EDI|NCL|LBA|GLA|BFS|MAN|BHX|STN|EMA,"[AYT, Antalya, AYT, Turkey]",True,antalya
7,Spain,Barcelona Terminal 2 BCN,BCN,True,True,BHX|MAN|LBA|GLA,"[BCN, Barcelona, Terminal, BCN, ski, Andorra, ...",True,barcelona


In [6]:
jet2.drop(
    columns=[
        'searchTerms',
        'airportUrlKey',
        'isEnabledForBooking',
        'country',
        'label',
    ],
    inplace=True
)

In [7]:
jet2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 2 to 104
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code                  79 non-null     object
 1   isDepartureAirport    79 non-null     bool  
 2   isDestinationAirport  79 non-null     bool  
 3   destinationIataCodes  79 non-null     object
dtypes: bool(2), object(2)
memory usage: 2.0+ KB


In [8]:
airport_column_info = pd.read_csv(
    base_path / 'data' / 'external' / 'airport_database_headers',
    index_col=['Field']
)


In [9]:
airport_column_info

,Name,Type
Field,,
1,ICAO Code,"String (3-4 chars, A - Z)"
2,IATA Code,"String (3 chars, A - Z)"
3,Airport Name,String
4,City/Town,String
5,Country,String
6,Latitude Degrees,"Integer [0,360]"
7,Latitude Minutes,"Integer [0,60]"
8,Latitude Seconds,"Integer [0,60]"
9,Latitude Direction,Char (N or S)


In [10]:
airport_columns = airport_column_info['Name'].to_list()

In [11]:
airports = pd.read_table(
     base_path / 'data' / 'external' / 'airport_database',
     sep=':',
     header=None
)
airports.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,AYGA,GKA,GOROKA,GOROKA,PAPUA NEW GUINEA,6,4,54,S,145,23,30,E,1610,-6.082,145.392
1,AYLA,LAE,NaN,LAE,PAPUA NEW GUINEA,0,0,0,U,0,0,0,U,0,0.000,0.000
2,AYMD,MAG,MADANG,MADANG,PAPUA NEW GUINEA,5,12,25,S,145,47,19,E,7,-5.207,145.789
3,AYMH,HGU,MOUNT HAGEN,MOUNT HAGEN,PAPUA NEW GUINEA,5,49,34,S,144,17,46,E,1643,-5.826,144.296
4,AYNZ,LAE,NADZAB,NADZAB,PAPUA NEW GUINEA,6,34,11,S,146,43,34,E,73,-6.570,146.726


In [12]:
airports.columns = airport_columns
airports.head()

,ICAO Code,IATA Code,Airport Name,City/Town,Country,Latitude Degrees,Latitude Minutes,Latitude Seconds,Latitude Direction,Longitude Degrees,Longitude Minutes,Longitude Seconds,Longitude Direction,Altitude,Latitude Decimal Degrees,Longitude Decimal Degrees
0,AYGA,GKA,GOROKA,GOROKA,PAPUA NEW GUINEA,6,4,54,S,145,23,30,E,1610,-6.082,145.392
1,AYLA,LAE,NaN,LAE,PAPUA NEW GUINEA,0,0,0,U,0,0,0,U,0,0.000,0.000
2,AYMD,MAG,MADANG,MADANG,PAPUA NEW GUINEA,5,12,25,S,145,47,19,E,7,-5.207,145.789
3,AYMH,HGU,MOUNT HAGEN,MOUNT HAGEN,PAPUA NEW GUINEA,5,49,34,S,144,17,46,E,1643,-5.826,144.296
4,AYNZ,LAE,NADZAB,NADZAB,PAPUA NEW GUINEA,6,34,11,S,146,43,34,E,73,-6.570,146.726


In [13]:
airports['City/Town'] = airports['City/Town'].str.title()
airports['Country'] = airports['Country'].str.title()
airports.head()

,ICAO Code,IATA Code,Airport Name,City/Town,Country,Latitude Degrees,Latitude Minutes,Latitude Seconds,Latitude Direction,Longitude Degrees,Longitude Minutes,Longitude Seconds,Longitude Direction,Altitude,Latitude Decimal Degrees,Longitude Decimal Degrees
0,AYGA,GKA,GOROKA,Goroka,Papua New Guinea,6,4,54,S,145,23,30,E,1610,-6.082,145.392
1,AYLA,LAE,NaN,Lae,Papua New Guinea,0,0,0,U,0,0,0,U,0,0.000,0.000
2,AYMD,MAG,MADANG,Madang,Papua New Guinea,5,12,25,S,145,47,19,E,7,-5.207,145.789
3,AYMH,HGU,MOUNT HAGEN,Mount Hagen,Papua New Guinea,5,49,34,S,144,17,46,E,1643,-5.826,144.296
4,AYNZ,LAE,NADZAB,Nadzab,Papua New Guinea,6,34,11,S,146,43,34,E,73,-6.570,146.726


In [14]:
sub_titles = ['Degrees', 'Minutes', 'Seconds', 'Direction']

airports.drop(
    columns=[
        *[f'Latitude {sub}' for sub in sub_titles],
        *[f'Longitude {sub}' for sub in sub_titles],
        'ICAO Code',
        'Airport Name',
    ],
    inplace=True
)

In [15]:
airports.rename(
    columns={
        'Latitude Decimal Degrees': 'Latitude',
        'Longitude Decimal Degrees': 'Longitude'
    },
    inplace=True
)

In [16]:
jet2 = jet2.merge(
    airports,
    left_on='code',
    right_on='IATA Code',
)
jet2.drop(columns=['IATA Code'], inplace=True)
jet2.head()

,code,isDepartureAirport,isDestinationAirport,destinationIataCodes,City/Town,Country,Altitude,Latitude,Longitude
0,ALC,True,True,BHX|MAN|NCL|LBA|GLA|BFS|EMA|STN|EDI,Alicante,Spain,44,38.282,-0.558
1,LEI,True,True,BHX|STN|MAN|GLA|NCL|EMA|LBA,Almeria,Spain,22,36.844,-2.370
2,AMS,True,True,LBA,Amsterdam,Netherlands,-3,52.309,4.764
3,AYT,True,True,EDI|NCL|LBA|GLA|BFS|MAN|BHX|STN|EMA,Antalya,Turkey,54,36.901,30.792
4,BCN,True,True,BHX|MAN|LBA|GLA,Barcelona,Spain,4,41.297,2.078


In [17]:
jet2['destinationIataCodes'] = jet2['destinationIataCodes'].apply(lambda x: x.split('|'))
jet2.head()

,code,isDepartureAirport,isDestinationAirport,destinationIataCodes,City/Town,Country,Altitude,Latitude,Longitude
0,ALC,True,True,"[BHX, MAN, NCL, LBA, GLA, BFS, EMA, STN, EDI]",Alicante,Spain,44,38.282,-0.558
1,LEI,True,True,"[BHX, STN, MAN, GLA, NCL, EMA, LBA]",Almeria,Spain,22,36.844,-2.370
2,AMS,True,True,[LBA],Amsterdam,Netherlands,-3,52.309,4.764
3,AYT,True,True,"[EDI, NCL, LBA, GLA, BFS, MAN, BHX, STN, EMA]",Antalya,Turkey,54,36.901,30.792
4,BCN,True,True,"[BHX, MAN, LBA, GLA]",Barcelona,Spain,4,41.297,2.078


In [18]:
jet2 = jet2.explode('destinationIataCodes')

In [19]:
jet2.head()

,code,isDepartureAirport,isDestinationAirport,destinationIataCodes,City/Town,Country,Altitude,Latitude,Longitude
0,ALC,True,True,BHX,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,MAN,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,NCL,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,LBA,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,GLA,Alicante,Spain,44,38.282,-0.558


In [20]:
columns = [
    'label',
    'code',
    'City/Town',
    'Country',
    'Altitude',
    'Latitude',
    'Longitude'
]
jet2.rename(
    columns={col: f'Departure_{col}' for col in columns},
    inplace=True
)

In [21]:
jet2.head()

,Departure_code,isDepartureAirport,isDestinationAirport,destinationIataCodes,Departure_City/Town,Departure_Country,Departure_Altitude,Departure_Latitude,Departure_Longitude
0,ALC,True,True,BHX,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,MAN,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,NCL,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,LBA,Alicante,Spain,44,38.282,-0.558
0,ALC,True,True,GLA,Alicante,Spain,44,38.282,-0.558


In [22]:
jet2 = jet2.merge(
    airports,
    left_on='destinationIataCodes',
    right_on='IATA Code',
    how='left'
)
jet2.head()

,Departure_code,isDepartureAirport,isDestinationAirport,destinationIataCodes,Departure_City/Town,Departure_Country,Departure_Altitude,Departure_Latitude,Departure_Longitude,IATA Code,City/Town,Country,Altitude,Latitude,Longitude
0,ALC,True,True,BHX,Alicante,Spain,44,38.282,-0.558,BHX,Birmingham,England,100.0,52.454,-1.748
1,ALC,True,True,MAN,Alicante,Spain,44,38.282,-0.558,MAN,Manchester,England,79.0,53.354,-2.275
2,ALC,True,True,NCL,Alicante,Spain,44,38.282,-0.558,NCL,Newcastle,England,82.0,55.038,-1.692
3,ALC,True,True,LBA,Alicante,Spain,44,38.282,-0.558,LBA,Leeds,England,208.0,53.866,-1.661
4,ALC,True,True,GLA,Alicante,Spain,44,38.282,-0.558,GLA,Glasgow,Uk,8.0,55.872,-4.433


In [23]:
jet2.rename(
    columns={col: f'Destination_{col}' for col in columns},
    inplace=True
)
jet2.head()

,Departure_code,isDepartureAirport,isDestinationAirport,destinationIataCodes,Departure_City/Town,Departure_Country,Departure_Altitude,Departure_Latitude,Departure_Longitude,IATA Code,Destination_City/Town,Destination_Country,Destination_Altitude,Destination_Latitude,Destination_Longitude
0,ALC,True,True,BHX,Alicante,Spain,44,38.282,-0.558,BHX,Birmingham,England,100.0,52.454,-1.748
1,ALC,True,True,MAN,Alicante,Spain,44,38.282,-0.558,MAN,Manchester,England,79.0,53.354,-2.275
2,ALC,True,True,NCL,Alicante,Spain,44,38.282,-0.558,NCL,Newcastle,England,82.0,55.038,-1.692
3,ALC,True,True,LBA,Alicante,Spain,44,38.282,-0.558,LBA,Leeds,England,208.0,53.866,-1.661
4,ALC,True,True,GLA,Alicante,Spain,44,38.282,-0.558,GLA,Glasgow,Uk,8.0,55.872,-4.433


In [24]:
jet2.drop(
    columns=['IATA Code'],
    inplace=True
)
jet2.head()

,Departure_code,isDepartureAirport,isDestinationAirport,destinationIataCodes,Departure_City/Town,Departure_Country,Departure_Altitude,Departure_Latitude,Departure_Longitude,Destination_City/Town,Destination_Country,Destination_Altitude,Destination_Latitude,Destination_Longitude
0,ALC,True,True,BHX,Alicante,Spain,44,38.282,-0.558,Birmingham,England,100.0,52.454,-1.748
1,ALC,True,True,MAN,Alicante,Spain,44,38.282,-0.558,Manchester,England,79.0,53.354,-2.275
2,ALC,True,True,NCL,Alicante,Spain,44,38.282,-0.558,Newcastle,England,82.0,55.038,-1.692
3,ALC,True,True,LBA,Alicante,Spain,44,38.282,-0.558,Leeds,England,208.0,53.866,-1.661
4,ALC,True,True,GLA,Alicante,Spain,44,38.282,-0.558,Glasgow,Uk,8.0,55.872,-4.433
